In [1]:
import pandas as pd 
import numpy as np
from multiprocessing import cpu_count, Pool
import collections
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook

In [2]:
def parallelize(data, func, partitions):
    data_split = np.array_split(data, partitions)
    cores = cpu_count()
    pool = Pool(cores)
    result = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return result

In [10]:
train_1 = pd.read_csv('./Data/train_2016_v2.csv',
                     parse_dates = ['transactiondate'])

In [11]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Data columns (total 3 columns):
parcelid           90275 non-null int64
logerror           90275 non-null float64
transactiondate    90275 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 2.1 MB


In [12]:
train_1.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [11]:
properties_1 = pd.read_csv('./Data/properties_2016.csv')

/Users/Zhengyi/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
properties_1.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


### Merge two data sets into one

##### check duplication

In [13]:
np.sum(train_1.groupby('parcelid').agg({'parcelid':pd.Series.nunique}))

parcelid    90150
dtype: int64

In [14]:
len(train_1)

90275

In [15]:
train_1_temp = train_1.groupby('parcelid').count()

In [16]:
train_1_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 10711738 to 162960842
Data columns (total 2 columns):
logerror           90150 non-null int64
transactiondate    90150 non-null int64
dtypes: int64(2)
memory usage: 2.1 MB


In [17]:
train_1_temp_1 = train_1[train_1['parcelid'].isin(train_1_temp[train_1_temp['logerror']>1].reset_index()['parcelid'])]

In [18]:
properties_1[properties_1['parcelid'].isin(train_1_temp_1['parcelid'])]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
19186,11602482,1.0,NaN,NaN,5.0,5.0,NaN,1.0,5.0,NaN,...,NaN,NaN,1042338.0,2078707.0,2015.0,1036369.0,25440.05,NaN,NaN,6.037270e+13
25098,13850164,13.0,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,1.0,NaN,76740.0,260828.0,2015.0,184088.0,3079.84,NaN,NaN,6.059001e+13
40016,13859028,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,2.0,NaN,146280.0,693258.0,2015.0,546978.0,8004.58,NaN,NaN,6.059099e+13
69420,13041169,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,...,NaN,NaN,74606.0,139709.0,2015.0,65103.0,1869.97,NaN,NaN,6.037401e+13
81799,12448490,1.0,NaN,NaN,3.0,3.0,NaN,4.0,3.0,NaN,...,NaN,NaN,186600.0,285000.0,2015.0,98400.0,3554.12,NaN,NaN,6.037570e+13
87242,14613416,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,104032.0,287254.0,2015.0,183222.0,3439.08,NaN,NaN,6.059099e+13
118872,14367791,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,2255544.0,4074029.0,2015.0,1818485.0,43143.70,NaN,NaN,NaN
119515,17151530,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,1.0,NaN,44843.0,56017.0,2015.0,11174.0,634.14,NaN,NaN,6.111004e+13
128638,12978851,NaN,NaN,NaN,3.0,6.0,NaN,7.0,3.0,NaN,...,NaN,NaN,94562.0,202988.0,2015.0,108426.0,3223.35,NaN,NaN,6.037405e+13
140622,12276495,NaN,NaN,NaN,3.0,4.0,NaN,7.0,3.0,NaN,...,NaN,NaN,156001.0,217717.0,2015.0,61716.0,3885.56,NaN,NaN,6.037543e+13


##### There are duplicated parcelid in train_1 data, only keep the one with most current transaction date

In [19]:
train_2 = train_1.sort_values(['parcelid','transactiondate'], ascending=[True, False]).drop_duplicates('parcelid', keep = 'first')

In [20]:
### check duplication again
np.sum(train_2.groupby('parcelid').agg({'parcelid':pd.Series.nunique}))

parcelid    90150
dtype: int64

In [21]:
len(train_2)

90150

In [22]:
prop_train = pd.merge(train_2, properties_1,
                           left_on = 'parcelid',
                           right_on = 'parcelid',
                           how = 'inner')

In [23]:
### check number of parcelid marched
prop_train['parcelid'].nunique()

90150

In [24]:
### create year
prop_train['year'] = prop_train['transactiondate'].dt.year

In [25]:
### create month
prop_train['month'] = prop_train['transactiondate'].dt.month

In [26]:
### create yearmonth
prop_train['yearmonth'] = prop_train['transactiondate'].dt.year*100+prop_train['transactiondate'].dt.month

In [27]:
prop_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 0 to 90149
Data columns (total 63 columns):
parcelid                        90150 non-null int64
logerror                        90150 non-null float64
transactiondate                 90150 non-null datetime64[ns]
airconditioningtypeid           28748 non-null float64
architecturalstyletypeid        260 non-null float64
basementsqft                    43 non-null float64
bathroomcnt                     90150 non-null float64
bedroomcnt                      90150 non-null float64
buildingclasstypeid             16 non-null float64
buildingqualitytypeid           57284 non-null float64
calculatedbathnbr               88974 non-null float64
decktypeid                      658 non-null float64
finishedfloor1squarefeet        6850 non-null float64
calculatedfinishedsquarefeet    89492 non-null float64
finishedsquarefeet12            85485 non-null float64
finishedsquarefeet13            33 non-null float64
finishedsquarefeet15

#### Define function to check categorical variable

In [180]:
class categoryvar(object):
    
    def __init__(self, ind_var, dep_var, input_data, dep_continuous=True):
        self.ind_var = ind_var
        self.dep_var = dep_var
        self.input_data = input_data
        self.dep_continuous = dep_continuous
        
        self.temp = self.input_data[self.input_data[self.ind_var].notnull()]
        if len(self.input_data)>len(self.temp):
            self.temp = self.input_data.copy()
            self.input_data[self.ind_var].fillna('Missing value', inplace = True)

    def basic_stat(self):
        if self.dep_continuous==True:
            basic_stat_df = self.temp.groupby(self.ind_var).agg({self.dep_var: {'Count':'count',
                                                                                'Total': np.sum,
                                                                                'Mean': np.mean,
                                                                                'Max': np.max,
                                                                                'Min': np.min,
                                                                                'Median': np.median,
                                                                                'Std': np.std}})
            basic_stat_df.columns = ['_'.join(x) for x in basic_stat_df.columns.ravel()]
            basic_stat_df = basic_stat_df.reset_index()
        else:
            cnt = pd.crosstab(temp[self.dep_var], temp[self.ind_var], margins = True).reset_index()
            row = pd.crosstab(temp[self.dep_var], temp[self.ind_var], normalize= 'index').reset_index()
            col = pd.crosstab(temp[self.dep_var], temp[self.ind_var], normalize = 'columns').reset_index()
            al = pd.crosstab(temp[self.dep_var], temp[self.ind_var], normalize = 'all').reset_index()
            basic_stat_df = ((cnt.append(row)).append(col)).append(al)
            
        return basic_stat_df
    
    def stat_test(self):
        from statsmodels.formula.api import ols
        if self.dep_continuous==True:
            f_oneway = ols('{}~{}'.format(self.dep_var, self.ind_var), data = self.temp).fit()
            stat_test_result = "P-value for F Test:{}".format(f_oneway.f_pvalue)
            
        else:
            
        return print(stat_test_result)
            
            
            
    
            

In [163]:
cate_len = temp['airconditioningtypeid'].unique()

In [167]:
from statsmodels.formula.api import ols

In [169]:
t = ols(formula = "logerror~airconditioningtypeid", data = temp).fit()

In [ ]:
t.f_pvalue

In [162]:
temp.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,year,month,yearmonth
0,10711738,0.0276,2016-08-02,1,NaN,NaN,3.0,4.0,NaN,4.0,...,567112.0,2015.0,321932.0,7219.18,NaN,NaN,6.037113e+13,2016,8,201608
1,10711755,-0.0182,2016-08-02,1,NaN,NaN,3.0,3.0,NaN,4.0,...,459844.0,2015.0,205153.0,6901.09,NaN,NaN,6.037113e+13,2016,8,201608
2,10711805,-0.1009,2016-05-03,1,NaN,NaN,2.0,3.0,NaN,4.0,...,384787.0,2015.0,149673.0,4876.61,NaN,NaN,6.037113e+13,2016,5,201605
3,10711816,-0.0121,2016-04-05,1,NaN,NaN,2.0,4.0,NaN,4.0,...,437176.0,2015.0,174867.0,5560.07,NaN,NaN,6.037113e+13,2016,4,201604
4,10711858,-0.0481,2016-07-15,1,NaN,NaN,2.0,4.0,NaN,4.0,...,382055.0,2015.0,150018.0,4878.25,NaN,NaN,6.037113e+13,2016,7,201607


In [181]:
t2 = categoryvar('airconditioningtypeid', 'logerror', prop_train,dep_continuous=True)

In [176]:
t3 = categoryvar('airconditioningtypeid', 'month', prop_train,dep_continuous=False)

In [148]:
cnt = pd.crosstab(temp['month'],temp['bedroomcnt']).reset_index()

In [149]:
row = pd.crosstab(temp['month'],temp['bedroomcnt'], normalize= 'index').reset_index()

In [150]:
col = pd.crosstab(temp['month'],temp['bedroomcnt'], normalize= 'columns').reset_index()

In [153]:
al = pd.crosstab(temp['month'],temp['bedroomcnt'], normalize= 'all').reset_index()

In [137]:
t1.basic_stat()

,bedroomcnt,logerror_Count,logerror_Total,logerror_Mean,logerror_Max,logerror_Min,logerror_Median,logerror_Std
0,0.0,1414,13.7336,0.009713,2.5600,-2.6880,0.00200,0.279360
1,1.0,3892,43.4035,0.011152,2.9530,-1.2980,0.00700,0.150873
2,2.0,22327,120.9029,0.005415,4.7370,-3.1940,0.00400,0.165527
3,3.0,35402,376.9891,0.010649,3.4430,-4.6050,0.00400,0.149518
4,4.0,20255,316.8017,0.015641,4.5200,-2.3540,0.00700,0.155504
5,5.0,5068,124.3119,0.024529,3.4030,-2.3650,0.01390,0.171709
6,6.0,1116,24.7238,0.022154,2.2920,-0.9014,0.01290,0.214295
7,7.0,234,12.1841,0.052069,2.4500,-0.7508,0.02660,0.299486
8,8.0,273,5.0995,0.018679,1.7490,-0.7941,0.01090,0.205894
9,9.0,91,1.9324,0.021235,1.5470,-0.4201,0.00700,0.218736


In [150]:
def category_explore(ind_var, dep_var, input_data, dep_continuous=True):
        
    temp = input_data[input_data[ind_var].notnull()]
    
    if len(input_data)>len(temp):
        temp = input_data
        temp[indv_var].fillna('Missing value', inplace = True)
    
    if dep_continuous == True:
        t2 = temp.groupby(ind_var).agg({dep_var: {'Count':'count',
                                                      'Total':np.sum,
                                                      'Mean': np.mean,
                                                      'Max': np.max,
                                                      'Min': np.min,
                                                      'Median': np.median,
                                                      'Std': np.std}})
        t2.columns = ['_'.join(x) for x in t2.columns.ravel()]
        t2 = t2.reset_index()
        

            

        
        
        
            
            
        
    return t2, temp

            
            
            
    
    """
    t1 contains counts and percentage for the categorical variable if dependent variable is discrete;
    
    t2 contains counts and percentage for the categorical variable by discrete dependent variable;
    or mean, min, max, std, median for continuous dependent variable.
    """ 
    

In [151]:
t2, temp = category_explore('airconditioningtypeid', 'logerror', prop_train, dep_continuous=True)

In [152]:
t2

,airconditioningtypeid,logerror_Count,logerror_Total,logerror_Mean,logerror_Max,logerror_Min,logerror_Median,logerror_Std
0,1,26640,344.8079,0.012943,4.7370,-3.1940,0.0070,0.143326
1,3,1,0.0917,0.091700,0.0917,0.0917,0.0917,NaN
2,5,215,3.3717,0.015682,1.2210,-0.2370,0.0020,0.105728
3,9,1,0.0100,0.010000,0.0100,0.0100,0.0100,NaN
4,11,63,1.6565,0.026294,0.2111,-0.0987,0.0227,0.065106
5,13,1828,28.5751,0.015632,3.2890,-2.6880,0.0070,0.143074
6,Missing value,61402,660.5389,0.010758,4.5200,-4.6050,0.0050,0.168745


In [154]:
dep_var = 'logerror'
ind_var = 'airconditioningtypeid'

##### airconditioningtypeid 